In [3]:
# ============================
# SETUP
# ============================
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from torchsummary import summary
from PIL import Image

# ----------------------------
# Reproducibility
# ----------------------------
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cpu')

In [4]:
train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

test_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [5]:
train_dataset = datasets.ImageFolder("data/train", transform=train_transforms)
validation_dataset = datasets.ImageFolder("data/test", transform=test_transforms)

train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=20, shuffle=False)


In [6]:
class HairCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=0)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2,2)
        self.fc1 = nn.Linear(32*99*99, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = self.conv(x)
        x = self.relu(x)
        x = self.pool(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x   # BCEWithLogitsLoss expects raw logits


In [7]:
model = HairCNN().to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr=0.002, momentum=0.8)


In [8]:
summary(model, input_size=(3, 200, 200))

total_params = sum(p.numel() for p in model.parameters())
print("Total parameters:", total_params)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 198, 198]             896
              ReLU-2         [-1, 32, 198, 198]               0
         MaxPool2d-3           [-1, 32, 99, 99]               0
            Linear-4                   [-1, 64]      20,072,512
              ReLU-5                   [-1, 64]               0
            Linear-6                    [-1, 1]              65
Total params: 20,073,473
Trainable params: 20,073,473
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.46
Forward/backward pass size (MB): 21.54
Params size (MB): 76.57
Estimated Total Size (MB): 98.57
----------------------------------------------------------------
Total parameters: 20073473


In [9]:
num_epochs = 10
history = {"acc": [], "loss": [], "val_acc": [], "val_loss": []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.float().unsqueeze(1).to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct / total
    history["loss"].append(epoch_loss)
    history["acc"].append(epoch_acc)

    # ----- Validation -----
    model.eval()
    val_loss = 0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.float().unsqueeze(1).to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_loss /= len(validation_dataset)
    val_acc = val_correct / val_total

    history["val_loss"].append(val_loss)
    history["val_acc"].append(val_acc)

    print(f"Epoch {epoch+1}/{num_epochs}  "
          f"Loss:{epoch_loss:.4f} Acc:{epoch_acc:.4f}  "
          f"ValLoss:{val_loss:.4f} ValAcc:{val_acc:.4f}")


Epoch 1/10  Loss:0.6665 Acc:0.6112  ValLoss:0.6511 ValAcc:0.6617
Epoch 2/10  Loss:0.5702 Acc:0.6787  ValLoss:0.6332 ValAcc:0.6318
Epoch 3/10  Loss:0.5207 Acc:0.7350  ValLoss:0.6143 ValAcc:0.6766
Epoch 4/10  Loss:0.4773 Acc:0.7600  ValLoss:0.6049 ValAcc:0.6617
Epoch 5/10  Loss:0.4606 Acc:0.7550  ValLoss:0.7307 ValAcc:0.5672
Epoch 6/10  Loss:0.3954 Acc:0.8275  ValLoss:0.6412 ValAcc:0.6866
Epoch 7/10  Loss:0.2844 Acc:0.8838  ValLoss:0.8307 ValAcc:0.6816
Epoch 8/10  Loss:0.2885 Acc:0.8788  ValLoss:0.7052 ValAcc:0.7114
Epoch 9/10  Loss:0.1882 Acc:0.9313  ValLoss:0.9275 ValAcc:0.6866
Epoch 10/10  Loss:0.2585 Acc:0.8912  ValLoss:0.8158 ValAcc:0.6915


In [10]:
import statistics

print("Median Training Accuracy:", statistics.median(history["acc"]))
print("Std Dev Train Loss:", statistics.stdev(history["loss"]))


Median Training Accuracy: 0.79375
Std Dev Train Loss: 0.15408442047389098


In [11]:
aug_transforms = transforms.Compose([
    transforms.RandomRotation(50),
    transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

train_dataset_aug = datasets.ImageFolder("data/train", transform=aug_transforms)
train_loader_aug = DataLoader(train_dataset_aug, batch_size=20, shuffle=True)


In [ ]:
aug_history = {"val_loss": [], "val_acc": []}

for epoch in range(10):
    model.train()
    for images, labels in train_loader_aug:
        images, labels = images.to(device), labels.float().unsqueeze(1).to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    model.eval()
    total_loss = 0
    total_correct = 0
    total_samples = 0
    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.float().unsqueeze(1).to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_correct += (predicted == labels).sum().item()
            total_samples += labels.size(0)

    avg_loss = total_loss / total_samples
    avg_acc = total_correct / total_samples

    aug_history["val_loss"].append(avg_loss)
    aug_history["val_acc"].append(avg_acc)

    print(f"Aug Epoch {epoch+1}/10  "
          f"ValLoss:{avg_loss:.4f}  ValAcc:{avg_acc:.4f}")


Aug Epoch 1/10  ValLoss:0.5963  ValAcc:0.7114
Aug Epoch 2/10  ValLoss:0.5968  ValAcc:0.7214
Aug Epoch 3/10  ValLoss:0.5870  ValAcc:0.7015
Aug Epoch 4/10  ValLoss:0.5696  ValAcc:0.7264
